#  Classification : Nearest Neighbors and Naive Bayes 

## Classification using Nearest Neighbors

(a) Perform k-Nearest neighbours on the given dataset($X_{knn}$ and $y_{knn}$: where $X_{knn}$ stores feature vectors representing the movies and  $y_{knn}$ stores the 0-1 labelling for each movie) for binary classification of movies, for classifiying whether a given movie is a comedy(label 1) or not a comedy(label 0) . Split the dataset into train(80%), validation(10%) and test sets(10%).Run k-Nearest neighbours for different k values (1,3,7,15,31,63). Select the k, using validation set, which returns the best accuracy score. 

(i)  Report all the validation accuracies for all the values of k. 
<br>(ii) Report accuracy score by performing k-NN on the test dataset using the best chosen k value. 

In [1]:
# import necessacy libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## write your code here.
X=pd.read_csv('X_knn.csv',header=None,sep=" ")
y=pd.read_csv('y_knn.csv',header=None)
X_train,X_tr_test,y_train,y_tr_test=train_test_split(X,y,test_size=0.2,random_state=23)
X_val,X_test,y_val,y_test=train_test_split(X_tr_test,y_tr_test,test_size=0.5,random_state=23)

In [ ]:
k_values=[1,3,7,15,31,63]
preds=np.zeros((len(X_val),len(k_values)))
i=0
for row in X_val.values:
    temp=pd.DataFrame(((X_train-np.reshape(row,(1,-1)))**2).sum(axis=1))
    temp['label']=y_train
    temp=temp.sort_values(by=[0])
    j=0
    for k in k_values:
        k_temp=temp[:k]
        if sum(k_temp['label'])>(k//2)+1:
            preds[i][j]=1
        else:
            preds[i][j]=0
        j+=1
    i+=1        

In [ ]:
for i in range(len(k_values)):
    print('k:',k_values[i],'=>',accuracy_score(y_val,preds[:,i]))
accuracies=[accuracy_score(y_val,preds[:,i]) for i in range(len(k_values))]
plt.plot(k_values,accuracies)
plt.title('K vs validation accuracy plot')
plt.xlabel('K');plt.ylabel('Validation Accuracy')
plt.show()

print("optimal value of k : ",k_values[accuracies.index(max(accuracies))])

In [ ]:
preds_test=np.zeros((len(X_test),1))
i=0
for row in X_test.values:
    temp=pd.DataFrame(((X_train-np.reshape(row,(1,-1)))**2).sum(axis=1))
    temp['label']=y_train
    temp=temp.sort_values(by=[0])
    j=0
    
    k_temp=temp[:optimal_k]
    if sum(k_temp['label'])>(optimal_k//2)+1:
        preds_test[i][0]=1
    else:
        preds_test[i][0]=0
    i+=1

print('k:',optimal_k,' => Test Result :',accuracy_score(y_test,preds_test))

Write the results here

(b) State why using an even value of k in k-NN should not be chosen

Write answer here

## Learning Naive Bayes' classifier  

### From Continuous Distribution of data

Here, the distribution of the data( $X$ represents the datapoints and $Y$ represents the 0-1 binary-class label; where 0 being the negative class and 1 being the positive class) is already known.
<br>Consider the following one-dimensional(1-D) Gaussian distributions where means and variances are unknown. You need to estimate means($\mu_-$: for negative class and  $\mu_+$: for positive class) and variances ($\sigma^{2}_{-}$: for negative class and $\sigma^{2}_+$: for positive class) from the given data : 
<br> (1) Assume $X|Y_{Y=0} \sim \mathcal{N}(\mu_- , \sigma^{2}_-)$ 
<br>(2) Assume $X|Y_{Y=1} \sim \mathcal{N}(\mu_+ , \sigma^{2}_+)$


*Generating artificial datasets in the next cell *

In [ ]:
## This cell is for generating datasets. Students should not change anything in this cell. 
## You can compare your mean and variance estimates by the actual ones used to generate these datasets

import numpy as np
X_pos = np.random.randn(1000,1)+np.array([[2.]])
X_neg = np.random.randn(1000,1)+np.array([[4.]])
X_train_pos = X_pos[:900]
X_train_neg = X_neg[:900]
X_test_pos = X_pos[900:]
X_test_neg = X_neg[900:]
X_train = np.concatenate((X_train_pos, X_train_neg), axis=0)
X_test = np.concatenate((X_test_pos, X_test_neg), axis=0)
Y_train = np.concatenate(( np.ones(900),np.zeros(900) ))
Y_test = np.concatenate(( np.ones(100), np.zeros(100) ))

## X_train, X_test, Y_train, Y_test are your datasets to work with ####



<br>**Instructions to follow for learning a Baeysian classifier:** *(Code the formulae for estimating the different parameters yourself)*
<br> a)Utilize the training dataset to estimate the means($\hat{\mu_+}$,$\hat{\mu_-}$) and variances($\hat{\sigma^{2}_+}$, $\hat{\sigma^{2}_-}$) for both positive and negative classes  
b)Estimate the prior probability: $P(Y=1)$  ⟶ which could be referred to as: $\hat{a}$ 
<br>c)Estimate the classifier funtion/posterior probability:  $P(Y=1|X = x)$  ⟶ which could be referred to as $\hat{\eta(x)}$
<br>d)Find out the threshold value($x^*$) for classification by equating the estimated classifier function($\hat{\eta(x)}$)  with threshold probability of 0.5
<br>e)Classify the test dataset into the two classes using this threshold value($x^*$) and find out the **accuracy** of the prediction 

Return back:  $\hat{\mu_+}$, $\hat{\mu_-}$, $\hat{\sigma^{2}_+}$, $\hat{\sigma^{2}_-}$, $\hat{a}$, $x^*$ and accuracy from the code written 

*Hint: $X|Y_{Y=0} \sim \mathcal{N}(\mu_- , \sigma^{2}_-)$ implies $P_{X|Y=0} = \mathcal{N}(\mu_- , \sigma^{2}_-) $*


In [ ]:
def gauss_dens(x,mean,var):
    return (1/np.sqrt(2*np.pi*var))*np.exp(-0.5*((x-mean)/var)**2)
def classifier_function(x,mean_0,var_0,mean_1,var_1,prior_1):
    return (gauss_dens(x,mean_1,var_1)*prior_1)/(gauss_dens(x,mean_1,var_1)*prior_1+gauss_dens(x,mean_0,var_0)*(1-prior_1))

In [ ]:
## write your code here.  
threshold=0.5
mean_1=np.mean(X_train[:900])
mean_0=np.mean(X_train[900:])
var_1=np.var(X_train[:900])
var_0=np.var(X_train[900:])
prior_1=len(X_train[:900])/float(len(X_train))
prior_0=len(X_train[900:])/float(len(X_train))

In [ ]:
x = np.linspace(-2,8,4000)
for k in x:
    if classifier_function(k,mean_0,var_0,mean_1,var_1,prior_1) < 0.5:
        break
x_star = k 

In [ ]:
y_pred = [1 if x<=x_star else 0 for x in X_test]
print('Mean for positive samples:',mean_1)
print('Mean for negative samples:',mean_0)
print('Variance for positive samples:',var_1)
print('Variance for negative samples:',var_0)
print('Prior probability for class 1:',prior_1)
print('Prior probability for class 0:',prior_0)
print('Threshold value of x:',x_star)
print('The accuracy of the Naive-Bayes classifier on test data:',accuracy_score(Y_test,y_pred))

Write results here

### From Discrete distribution of data

Unlike the first exercise for learning the Naive Bayes' classifier where we dealt with continuous distribution of data, here you need to work with discrete data, which means finding Probability Mass Distribution(PMF). 

Age  | Income | Status  | Buy
-----|--------|-------- |----
<=20 |  low   | students| yes
<=20 |  high  | students| yes
<=20 | medium | students| no
<=20 | medium | married | no
<=20 |  high  | married | yes
21-30|  low   | married | yes
21-30|  low   | married | no 
21-30| medium | students| no
21-30|  high  | students| yes
 >30 |  high  | married | no
 >30 |  high  | married | yes
 >30 | medium | married | yes
 >30 | medium | married | no
 >30 | medium | students| no
 
Consider the train dataset above. Take any random datapoint ($X_{i}$) where $X_{i} = (X_{i,1} = Age,X_{i,2} = Income,X_{i,3} = Status)$ and its corresponding label 

($Y_{i} = Buy$). A "yes" in Buy corresponds to label-1 and a "no" in Buy corresponds to label-0.

<br>**Instructions to follow for learning a Baeysian classifier:** *(Code the formulae for estimating the different parameters yourself)*
<br> a)Estimate the prior probability: $P(Y=1)$  ⟶ which could be referred to as: $\hat{a}$   
b)Estimate the likelihood for each feature:  $P(X_{i,j} = x |Y = y_{i})$, where $ i$=datapoint counter, $j \in \{1,2,3\}$ and $y_{i} \in \{0,1\}$ 
<br>c)Estimate the total likelihood: $P(X_{i} = x |Y = y_{i})$  
d)Calculate the posterior probability: $P(Y = 1|X_{i} = x_{test} )$ = $p_{test}$ where $x_{test} = (Age = 21-30, Income= medium, Status = married)$


Return back: $\hat{a}$, total likelihood and $p_{test}$ 


In [ ]:
## write your code here.
data={'age':['<=20','<=20','<=20','<=20','<=20','21-30','21-30','21-30','21-30','>30','>30','>30','>30','>30'],
   'income':['low','high','medium','medium','high','low','low','medium','high','high','high','medium','medium','medium'],
   'status':['students', 'students','students','married','married','married','married', 'students','students','married','married','married','married','students'],
   'buy':['yes', 'yes', 'no', 'no','yes', 'yes', 'no', 'no','yes','no','yes', 'yes', 'no', 'no']}

data=pd.DataFrame(data)
p_1 = len(data[data['buy']=='yes'])/len(data)
def probability(col_name, value, label=1):
    return len(data[(data[col_name] == value) & (data['buy']==label)])/len(data[data['buy']==label])

def likelihood(age, income, status, label):
    p_age= probab('age',age,label)
    p_income = probability('income',income,label)
    p_status = probability('status',status,label)
    return p_age * p_income * p_status

Write results here

In [ ]:
income_list=['high','low','medium']
labels_list=['yes','no']
status_list=['married','students']
age_list=['<=20','21-30','>30']

print("The prior probability P(Y=1):", p_1,'\n')

for income in income_list:
    for i in labels_list:
        print("Income: ", income, "Buying Status: ", i, "Likelihood: ", probability('income',income,i)) 
for age in age_list:
    for i in labels_list:
        print("Age: ", age, "Buying Status: ", i, "Likelihood: ", probability('age',age,i))
for stat in status_list:
    for i in labels_list:
        print("Maritial Status: ", stat, "Buying Status: ", i, "Likelihood: ", probability('status',stat,i))

In [ ]:
#Printing the total likelihoods for all possible conditions
for income in income_list:
    for age in age_list:
        for stat in status_list:
            for label in labels_list:
                print("Income: ", income,' ', "Age: ", age,' ',"Maritial Status: ", stat,' ',"Likelihood: ",likelihood(age, income, stat, label))

In [ ]:
def posterior_discrete(age,income,status,buy):
    p_age = len(data[data['age'] == age])/len(data)
    p_income = len(data[data['income'] == income])/len(data)
    p_status = len(data[data['status'] == status])/len(data)

    if buy=='yes':
        p = p_1 * likelihood(age,income,status,buy)/(p_age*p_income*p_status)
    if buy=='no':
        p = (1-p_1) * likelihood(age,income,status,buy)/(p_age*p_income*p_status)
    return p

print(f"Age: 21-30, Income: medium, Status: married, Posterior probability: {posterior_discrete('21-30','medium','married','yes')}")